# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770437007128                   -0.52    9.0
  2   -2.771691347746       -2.90       -1.32    1.0
  3   -2.771714480229       -4.64       -2.49    1.0
  4   -2.771714650643       -6.77       -3.21    1.0
  5   -2.771714712064       -7.21       -3.78    2.0
  6   -2.771714715030       -8.53       -4.51    1.0
  7   -2.771714715244       -9.67       -5.06    2.0
  8   -2.771714715249      -11.26       -6.05    1.0
  9   -2.771714715250      -12.57       -7.02    2.0
 10   -2.771714715250   +  -14.35       -7.10    1.0
 11   -2.771714715250      -14.35       -8.39    1.0


-0.0001345731922687352

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770545087569                   -0.52    9.0
  2   -2.771781098036       -2.91       -1.32    1.0
  3   -2.771801700456       -4.69       -2.46    1.0
  4   -2.771802008796       -6.51       -3.18    1.0
  5   -2.771802074330       -7.18       -4.30    2.0
  6   -2.771802074388      -10.24       -4.42    1.0
  7   -2.771802074472      -10.07       -5.26    1.0
  8   -2.771802074476      -11.49       -5.46    2.0
  9   -2.771802074476      -12.37       -6.23    1.0
 10   -2.771802074476      -13.22       -6.69    1.0
 11   -2.771802074476      -14.88       -7.26    2.0
 12   -2.771802074476   +  -14.21       -7.69    1.0
 13   -2.771802074476      -13.83       -8.61    1.0


0.01761222095546544

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345731922687352
Displaced dipole:  0.01761222095546544
Polarizability :   1.7746794147734175


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920643618e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551042398e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852762764140e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593818128e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787470987e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.273949138966e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.727354399093e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.727354399093e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.518931966593e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.191723696354e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.722772499235e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 5.991584405051e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 5.991584405051e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.